In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm

In [ ]:
x = torch.rand(5, 3)

In [ ]:
y = torch.rand(3, 2)

In [ ]:
x @ y

In [ ]:
t = torch.tensor([1, 2])

In [ ]:
t.numel()

In [ ]:
t.shape

In [ ]:
t.ndim

In [ ]:
x.T

In [ ]:
x.device

In [ ]:
x.dtype

In [ ]:
x.numpy()

In [ ]:
torch.from_numpy(np.array([1, 2, 3], dtype=float))

# Tensori sulla cpu

In [ ]:
x

In [ ]:
x.device

In [ ]:
x = x.to('cuda:0')

In [ ]:
z = x @ y

In [ ]:
z = x @ y.to('cuda')

In [ ]:
%%time

x = torch.rand(1000, 1000)
y = torch.rand(1000, 1000)

for _ in range(500):
    x @ y

In [ ]:
%%time

x = torch.rand(1000, 1000, device='cuda')
y = torch.rand(1000, 1000, device='cuda')

for _ in range(500):
    x @ y

# Differenziazione automatica

In [ ]:
x = torch.rand(2, 2, requires_grad=True)
s = torch.sum(x**10)

In [ ]:
s.backward()

In [ ]:
x.grad

# Dataset

In [ ]:
n = 100_000
X = np.random.random_sample((n, 2))*2*np.pi - np.pi
y = np.sin(X[:, 0] + X[:, 1]).reshape(-1, 1)

In [ ]:
m = 10_000
X_test = np.random.random_sample((n, 2))*2*np.pi - np.pi
y_test = np.sin(X_test[:, 0] + X_test[:, 1]).reshape(-1, 1)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(X[:, 0], X[: ,1], y.ravel())

# Rete neurale

In [ ]:
class FuncDataset(Dataset):

    def __init__(self, X, y):
        super().__init__()
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
dataset = FuncDataset(X, y)

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 32
ALPHA = 1e-3

In [ ]:
data_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
device = 'cpu'
w1 = torch.randn(2, 50, device=device, requires_grad=True)
b1 = torch.zeros(1, 50, device=device, requires_grad=True)
w2 = torch.randn(50, 1, device=device, requires_grad=True)
b2 = torch.zeros(1, 1, device=device, requires_grad=True)

params = [w1, b1, w2, b2]

In [ ]:
def forward(x, y):
    z1 = x@w1 + b1
    a1 = F.relu(z1)
    y_ = a1@w2 + b2
    loss = F.mse_loss(y, y_)
    return y_, loss

# Addestramento

In [ ]:
losses = []
for epoch in range(NUM_EPOCHS):
    for i, (x_batch, y_batch) in tqdm(enumerate(data_loader)):
        y_, loss = forward(x_batch, y_batch)
        losses.append(loss)
        for param in params:
            if param.grad is not None:
                param.grad.zero_()
        loss.backward()
        with torch.no_grad():
            for param in params:
                param -= ALPHA * param.grad
        if i-1 % 1000 == 0:
            print(torch.mean(torch.tensor(losses[-1000:])))

# Test

In [ ]:
y_test_, _ = forward(torch.tensor(X_test).float(), torch.tensor(y_test).float())

In [ ]:
fig = plt.figure(figsize=(20, 10))

ax_train = fig.add_subplot(121, projection='3d', title='train set')
ax_test = fig.add_subplot(122, projection='3d', title='test set')

ax_test.plot_trisurf(X_test[:, 0], X_test[: ,1], y_test_.detach().numpy().ravel())
ax_train.plot_trisurf(X[:, 0], X[: ,1], y.ravel())